In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
t = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/Train-1542865627584.csv")
b = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/Train_Beneficiarydata-1542865627584.csv")
i = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/Train_Inpatientdata-1542865627584.csv")
o = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/Train_Outpatientdata-1542865627584.csv")

tt = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/Test-1542969243754.csv")
tb = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/Test_Beneficiarydata-1542969243754.csv")
ti = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/Test_Inpatientdata-1542969243754.csv")
to = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/Test_Outpatientdata-1542969243754.csv")


(len(t), len(b), len(i), len(o))

In [ ]:
i.head()

In [ ]:
i.Provider.nunique(), o.Provider.nunique()

In [ ]:
o.head()

In [ ]:
df = pd.concat([i,o])
df = pd.merge(df, t, on="Provider", how="outer")
df = df.fillna(0)

df2 = pd.concat([ti,to])
df2 = pd.merge(df2, tt, on="Provider", how="outer")
df2 = df2.fillna(0)

In [ ]:
df

In [ ]:
df2

In [ ]:
df['PotentialFraud'].value_counts()

# Labeling categorical data

In [ ]:
df.head()

In [ ]:
df.info()

# Preparing the dataset

### ClaimID : This a unique id for each claim submitted.
### Bene_ID : contains Unique Id of the beneficiaries registered for insurance scheme.
### AttendingPhysician : column contains the id of the physicians who attended the patient.
### OperatingPhysician: column contains the id of the the physicians who operated on the patient.
### Provider : unique Id of the healthcare providers.
### InscClaimAmtReimbursed : Total amount paid to the claimant after settlement.
### ClaimStartDt , ClaimEndDt : These columns has the date on which the claims were submitted and the date when claims got settled respectively.
### AdmissionDt , DischargeDt : Date on which the patient got admitted in hospital and the date on which patient got discharged . Applicable for only inpatient data.

In [ ]:
cols = ['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'DischargeDt']

X = df[cols]
Y = df["PotentialFraud"].apply(lambda x: True if x == "Yes" else False)

In [ ]:
X.head()

In [ ]:
le = preprocessing.LabelEncoder()
X['BeneID'] = le.fit_transform(X['BeneID'])
mapping_BeneID = dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping_BeneID)

In [ ]:
le = preprocessing.LabelEncoder()
X['ClaimID'] = le.fit_transform(X['ClaimID'])
mapping_ClaimID = dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping_ClaimID)

In [ ]:
le = preprocessing.LabelEncoder()
X['ClaimStartDt'] = le.fit_transform(X['ClaimStartDt'])
mapping_ClaimStartDt = dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping_ClaimStartDt)

In [ ]:
le = preprocessing.LabelEncoder()
X['ClaimEndDt'] = le.fit_transform(X['ClaimEndDt'])
mapping_ClaimEndDt = dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping_ClaimEndDt)

In [ ]:
le = preprocessing.LabelEncoder()
X['Provider'] = le.fit_transform(X['Provider'])
mapping_Provider = dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping_ClaimEndDt)

In [ ]:
# le = preprocessing.LabelEncoder()
# X['AttendingPhysician'] = le.fit_transform(X['AttendingPhysician'].astype(str))
# mapping_AttendingPhysician = dict(zip(le.classes_, le.transform(le.classes_)))
# print(mapping_AttendingPhysician)

In [ ]:
le = preprocessing.LabelEncoder()
X['OperatingPhysician'] = le.fit_transform(X['OperatingPhysician'].astype(str))
mapping_OperatingPhysician = dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping_OperatingPhysician)

In [ ]:
le = preprocessing.LabelEncoder()
X['OtherPhysician'] = le.fit_transform(X['OtherPhysician'].astype(str))
mapping_OtherPhysician = dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping_OtherPhysician)

In [ ]:
le = preprocessing.LabelEncoder()
X['AdmissionDt'] = le.fit_transform(X['AdmissionDt'].astype(str))
mapping_AdmissionDt= dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping_AdmissionDt)

In [ ]:
le = preprocessing.LabelEncoder()
X['DischargeDt'] = le.fit_transform(X['DischargeDt'].astype(str))
mapping_DischargeDt = dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping_DischargeDt)

In [ ]:
X.head()

In [ ]:
test.head()

In [ ]:
X.info()

In [ ]:
X.shape, Y.shape

In [ ]:
Y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=101)

In [ ]:
X_train.shape, y_train.shape

# Training

In [ ]:
import xgboost as xgb
clf = xgb.XGBClassifier(n_jobs=12, n_estimators=200)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, auc, roc_curve


In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
accuracy_score(y_test,y_pred)

# Feature importance plot

In [ ]:
X_train.info()

In [ ]:
importance = clf.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
dtc=  DecisionTreeClassifier()
dtc.fit(X_train, y_train)

In [ ]:
y_pred = dtc.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))

In [ ]:
print( classification_report(y_test,y_pred))

In [ ]:
importance = dtc.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
import pickle


In [ ]:
Pkl_Filename = "Model_xgb.pkl"  
with open(Pkl_Filename, 'wb') as file: 
    pickle.dump(clf, file)

In [ ]:
with open(Pkl_Filename, 'rb') as file:  
    
    Model_xgb = pickle.load(file)

Model_xgb

In [ ]:
cols = ['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'DischargeDt']

test = df[cols]
# Y = df["PotentialFraud"].apply(lambda x: True if x == "Yes" else False)

In [ ]:
test.head()

In [ ]:
test.BeneID = test.BeneID.map(mapping_BeneID)
test.ClaimID =  test.ClaimID.map(mapping_ClaimID)
test.ClaimStartDt =  test.ClaimStartDt.map(mapping_ClaimStartDt)
test.ClaimEndDt =  test.ClaimEndDt.map(mapping_ClaimEndDt)
test.Provider =  test.Provider.map(mapping_Provider)
test.AttendingPhysician =  test.AttendingPhysician.map(mapping_AttendingPhysician)
test.OperatingPhysician =  test.OperatingPhysician.map(mapping_OperatingPhysician)
test.OtherPhysician =  test.OtherPhysician.map(mapping_OtherPhysician)
test.AdmissionDt =  test.AdmissionDt.map(mapping_AdmissionDt)
test.DischargeDt =  test.DischargeDt.map(mapping_DischargeDt)


In [ ]:
test = test.fillna(0)

In [ ]:
test.head()

In [ ]:
test.loc[[1]]

In [ ]:
Model_xgb.predict(test.loc[[1]])